In [13]:
# ============================================
# ANÁLISIS GEOESPACIAL DE ACCIDENTES EN COSTA RICA (2023)
# ============================================

# === Dependencias ===
import pandas as pd
import numpy as np
from pathlib import Path
import folium
from unidecode import unidecode

# === Rutas ===
DATA_PATH = Path(r"C:\Users\98248\Downloads\PYCHAR\proyecto_accidentes\proyecto_accidentes\src\data\processed\accidentes_clima_2023.csv")
GEO_PATH = Path(r"C:\Users\98248\Downloads\PYCHAR\proyecto_accidentes\proyecto_accidentes\src\data\geo\provincias.geojson")

# Carpeta de salidas
reports_dir = Path("reports")
reports_dir.mkdir(parents=True, exist_ok=True)

# === Carga del dataset ===
if not DATA_PATH.exists():
    raise FileNotFoundError(f"No se encontró el CSV en: {DATA_PATH}")
if not GEO_PATH.exists():
    raise FileNotFoundError(f"No se encontró el GeoJSON en: {GEO_PATH}")

print("✅ Dataset:", DATA_PATH)
print("✅ GeoJSON:", GEO_PATH)

df = pd.read_csv(DATA_PATH, parse_dates=["fecha"])
df.head()


✅ Dataset: C:\Users\98248\Downloads\PYCHAR\proyecto_accidentes\proyecto_accidentes\src\data\processed\accidentes_clima_2023.csv
✅ GeoJSON: C:\Users\98248\Downloads\PYCHAR\proyecto_accidentes\proyecto_accidentes\src\data\geo\provincias.geojson


,fecha,hora,provincia,tipo_via,clase de accidente,tipo de accidente,estado del tiempo,ruta,kilómetro,precip_acum
0,2017-10-01,20,San José,Nacional,Solo heridos leves,Colisión con motocicleta,Lluvia,2,141,NaN
1,2017-01-03,6,Heredia,Cantonal,Solo heridos leves,Colisión con motocicleta,Buen tiempo,Cantonal,Cantonal,NaN
2,2017-01-03,16,Heredia,Nacional,Solo heridos leves,Atropello a persona,Buen tiempo,116,4,NaN
3,2017-01-05,18,Heredia,Nacional,Solo heridos leves,Colisión con motocicleta,Buen tiempo,106,4,NaN
4,2017-01-05,20,Heredia,Nacional,Solo heridos leves,Colisión con motocicleta,Buen tiempo,116,2,NaN


In [14]:
# === Normalización de nombres de provincia ===
def normalizar_prov(x: str) -> str:
    if pd.isna(x):
        return x
    t = unidecode(str(x)).strip().upper()
    mapa = {
        "SAN JOSE": "San José",
        "ALAJUELA": "Alajuela",
        "CARTAGO": "Cartago",
        "HEREDIA": "Heredia",
        "GUANACASTE": "Guanacaste",
        "PUNTARENAS": "Puntarenas",
        "LIMON": "Limón",
    }
    return mapa.get(t, str(x).strip())

print("Antes:", sorted(df["provincia"].dropna().unique()))
df["provincia"] = df["provincia"].apply(normalizar_prov)
print("Después:", sorted(df["provincia"].dropna().unique()))


Antes: ['Alajuela', 'Cartago', 'Guanacaste', 'Heredia', 'Limón', 'Puntarenas', 'San José']
Después: ['Alajuela', 'Cartago', 'Guanacaste', 'Heredia', 'Limón', 'Puntarenas', 'San José']


In [15]:
# === Mapa con círculos proporcionales ===
centroides = {
    "San José": (9.928, -84.090),
    "Alajuela": (10.016, -84.215),
    "Cartago": (9.864, -83.919),
    "Heredia": (10.004, -84.116),
    "Guanacaste": (10.635, -85.437),
    "Puntarenas": (9.976, -84.833),
    "Limón": (9.990, -83.033),
}

acc_prov = df.groupby("provincia").size().reset_index(name="accidentes")

m3 = folium.Map(location=[9.9, -84.2], zoom_start=7, tiles="cartodbpositron")

max_acc = acc_prov["accidentes"].max() if len(acc_prov) else 1
for _, row in acc_prov.iterrows():
    prov = row["provincia"]
    acc = int(row["accidentes"])
    if prov in centroides:
        lat, lon = centroides[prov]
        folium.CircleMarker(
            location=(lat, lon),
            radius=max(5, (acc / max_acc) * 25),
            popup=folium.Popup(f"{prov}: {acc} accidentes", max_width=240),
            color="crimson",
            fill=True,
            fill_opacity=0.6
        ).add_to(m3)

m3.save("reports/mapa_circulos_accidentes.html")
m3
